# Title

## Imports

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintext
from pipeline import *

## Importing the dataset

_Choose and import the dataset adapted for the attack. Here, we choose a small AES-128 dataset for plaintext recovery attack._

In [7]:
data = AESDatasetCiphertextPlaintext(128, 'small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

_Print the dimension information about the dataset. I the dataset size is too big, it is possible to shorten it._

In [8]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (189568, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (189568, 128)
===== Sample Shape: (128,)
===== Testing Labels Shape: (81243, 128)
===== Testing Samples Shape: (81243, 128)


## Creating the model

_Imports will depend on the needs for the desired model architecture._

In [9]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

_Add customization here._

In [13]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*8

# loss functions
loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# learning rates
learning_rate = 0.1

# can be a scheduled learning rate
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)

# other hyper-parameters
optimizer = Adam(learning_rate=learning_rate)
metrics = ['accuracy', 'binary_accuracy']
epochs = 3
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

_Add customization here._

In [14]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              132096    
                                                                 
 dense_5 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_6 (Dense)             (None, 128)               131200    
                                                                 
Total params: 1,312,896
Trainable params: 1,312,896
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

_This block mostly stays the same._

In [15]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/3
35/35 [==============================] - 5s 147ms/step - loss: 0.2654 - accuracy: 0.7219 - binary_accuracy: 0.7278 - val_loss: 0.2641 - val_accuracy: 0.7440 - val_binary_accuracy: 0.7359
Epoch 2/3
35/35 [==============================] - 5s 140ms/step - loss: 0.2659 - accuracy: 0.7439 - binary_accuracy: 0.7341 - val_loss: 0.2641 - val_accuracy: 0.7440 - val_binary_accuracy: 0.7359
Epoch 3/3
35/35 [==============================] - 5s 139ms/step - loss: 0.2659 - accuracy: 0.7439 - binary_accuracy: 0.7341 - val_loss: 0.2641 - val_accuracy: 0.7440 - val_binary_accuracy: 0.7359


### Testing
Here, we evaluate the neural network with the test data.

_Some customization is possible here._

In [16]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

17/17 [==============================] - 1s 56ms/step - loss: 0.2671 - accuracy: 0.7455 - binary_accuracy: 0.7329
Test loss: 0.26705867052078247
Test accuracy: 0.7455288171768188


_Several ways to test the model._

In [19]:
results, predictions = test_model(neural_network, test_samples, test_labels, batch_size, ascii_correction=True)

17/17 [==============================] - 1s 45ms/step


In [20]:
print("Correct bytes: " + str(results["correct_bytes"]))
print("Byte accuracy: " + str(results["byte_accuracy"]))

print("Correct predictions: " + str(results["correct_predictions"]))
print("Accuracy: " + str(results["accuracy"]))

Correct bytes: 5542
Byte accuracy: 0.004263444235195648
Correct predictions: 0
Accuracy: 0.0


### Prediction
Here is where we use the network as an attack. We could skip the testing phase and use this as our own testing phase. Here, we can also evaluate some result and compute other metrics.

_Some customization can be necessary._

In [29]:
size = 1000

In [30]:
predictions = [predict_sample(neural_network, test_samples[i]) for i in range(size)]

1/1 [==============================] - 0s 20ms/step


In [23]:
metrics = [correct_and_metrics((predictions[i], test_labels[i])) for i in range(size)]

In [24]:
correct_bytes = 0
correct_predictions = 0
for m in metrics:
    correct_bytes += m[0]
    correct_predictions += m[1]
                             
print("Correct bytes: {}".format(correct_bytes))
print("Byte accuracy: {}".format(correct_bytes/(2*size)))
print("Correct predictions: {}".format(correct_predictions))
print("Prediction accuracy: {}".format(correct_predictions/size))

Correct bytes: 76
Byte accuracy: 0.038
Correct predictions: 0
Prediction accuracy: 0.0


In [28]:
for i in range(size):
    correct = prediction_to_string(test_labels[i])
    predicted = prediction_to_string(predictions[i])
    print("correct-> " + correct + " | " + predicted + " <-predicted")

correct-> :33 And when thi | ````dd`````````` <-predicted
correct-> s cometh to pass | ````dd`````````` <-predicted
correct-> , (lo, it will c | ````dd`````````` <-predicted
